In [1]:
import sys
print(sys.executable)


C:\Users\KATANA\Desktop\Jupyter-Lab\workshop\slip_recognition\.env\Scripts\python.exe


In [2]:
!pip install opencv-python

In [3]:
import cv2
import json
import numpy as np
import os
import paddle
from paddleocr import PaddleOCR

C:\Users\KATANA\Desktop\Jupyter-Lab\workshop\slip_recognition\.env\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
C:\Users\KATANA\Desktop\Jupyter-Lab\workshop\slip_recognition\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Checking connectivity to the model hosters, this may take a while. To bypass this check, set `DISABLE_MODEL_SOURCE_CHECK` to `True`.


In [4]:
slip_template_image_path = r"../data/slip_template/slip_template.jpg"

In [5]:
IMAGE_PATH = slip_template_image_path
OUTPUT_JSON = "roi_config.json"

def imread_unicode(path):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

img = imread_unicode(IMAGE_PATH)
if img is None:
    raise FileNotFoundError("Cannot load image")

clone = img.copy()
drawing = False
start_point = None
rois = {}

def mouse_callback(event, x, y, flags, param):
    global drawing, start_point, img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img = clone.copy()
        cv2.rectangle(img, start_point, (x, y), (0, 255, 0), 2)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_point = (x, y)
        cv2.rectangle(img, start_point, end_point, (0, 255, 0), 2)

        name = input("Enter ROI name (amount / fee / transaction_id): ").strip()
        if name:
            rois[name] = {
                "x1": min(start_point[0], end_point[0]),
                "y1": min(start_point[1], end_point[1]),
                "x2": max(start_point[0], end_point[0]),
                "y2": max(start_point[1], end_point[1])
            }
            print(f"✅ Saved ROI: {name}")

cv2.namedWindow("Select Multiple ROI")
cv2.setMouseCallback("Select Multiple ROI", mouse_callback)

print("🖱️ Drag to select ROI")
print("⌨️ Type ROI name in terminal")
print("💾 Press S to save all | Q to quit")

while True:
    cv2.imshow("Select Multiple ROI", img)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("s"):
        with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
            json.dump(rois, f, indent=2, ensure_ascii=False)
        print(f"✅ All ROIs saved to {OUTPUT_JSON}")
        break

    elif key == ord("q"):
        break

cv2.destroyAllWindows()

🖱️ Drag to select ROI
⌨️ Type ROI name in terminal
💾 Press S to save all | Q to quit


Enter ROI name (amount / fee / transaction_id):  datetime_ transaction


✅ Saved ROI: datetime_ transaction


Enter ROI name (amount / fee / transaction_id):  account_name


✅ Saved ROI: account_name


...

✅ Saved ROI: transaction_id


Enter ROI name (amount / fee / transaction_id):  amount


✅ Saved ROI: amount


Enter ROI name (amount / fee / transaction_id):  transfer_fee


✅ Saved ROI: transfer_fee
✅ All ROIs saved to roi_config.json


In [6]:
ocr = PaddleOCR(
    lang="th",                                # Thai language model
    device="gpu",
    ocr_version="PP-OCRv5",                   # Latest OCR pipeline
    use_doc_orientation_classify=False,       # Skip document orientation (not needed for receipts)
    use_textline_orientation=False,           # Skip textline orientation (Thai is mostly horizontal)
    use_doc_unwarping=False,                  # Skip unwarping (receipt is already flat)
    text_det_limit_side_len=960,              # Resize long side to 960px for better detection
    text_det_limit_type="max",                # Limit resizing by max side
    text_det_thresh=0.3,                      # Lower threshold for small text boxes
    text_det_box_thresh=0.5,                  # Box confidence threshold
    text_det_unclip_ratio=2.0,                # Expand detected boxes slightly
    text_rec_score_thresh=0.5,                # Recognition confidence threshold
    return_word_box=True                      # Return word-level boxes
)


Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\KATANA\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('th_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\KATANA\.paddlex\official_models\th_PP-OCRv5_mobile_rec`.


In [7]:
print("Is GPU available?", paddle.is_compiled_with_cuda())
print("Current device:", paddle.get_device())

Is GPU available? True
Current device: gpu:0


In [9]:
def imread_unicode(path):
    data = np.fromfile(path, dtype=np.uint8)
    return cv2.imdecode(data, cv2.IMREAD_COLOR)

img = imread_unicode("../data/raw/slip_example.jpg")

In [10]:
with open("roi_config.json", "r", encoding="utf-8") as f:
    rois = json.load(f)

results = {}

for name, box in rois.items():
    crop = img[box["y1"]:box["y2"], box["x1"]:box["x2"]]
    ocr_result = ocr.predict(crop)
    results[name] = ocr_result[0]["rec_texts"]

print(results)


{'datetime_ transaction': ['26 ม.ค. 65 05:10 น.'], 'account_name': ['นายกสิกร รักไทย'], 'destination account': ['ธ.กสิกรไทย'], 'account_number': ['888-8-8888-8'], 'transaction_id': ['123456789012345678'], 'amount': ['888.00 บาท'], 'transfer_fee': ['0.00 บาn']}
